<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/rclone_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📌 **Install**


In [ ]:
# Cài đặt rClone
!sudo -v ; curl https://rclone.org/install.sh | sudo bash

In [ ]:
# Danh sách remotes đang có
!rclone listremotes

In [ ]:
# Khôi phục file config
!cp "/content/drive/MyDrive/Colab Notebooks/rclone.conf" /root/.config/rclone/rclone.conf

# ⚙️ **Config remotes**

In [ ]:
# Tạo mới, chỉnh sửa, xóa hoặc kiểm tra remote.
!rclone config

# 💾 **Backup file config**

In [ ]:
# Backup config
!cp /root/.config/rclone/rclone.conf /content/rclone.conf
!rclone copy -v "/content/rclone.conf" "/content/drive/MyDrive/Colab Notebooks/backup/rclone*.conf"

# 🎯 **Các lệnh Copy**

In [ ]:
# @title Basic: Copy files between Cloud Remotes
# @markdown ### Hướng dẫn:
# @markdown - **Source Remote**: `remoteName:folder/`or `/content/drive/MyDrive/CBZ/image.jpg` .
# @markdown - **Destination Remote**:  `remoteName:folder/`.
source_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/","haocenter","tsd06","xuhao","archive"]
input_folder = "" # @param {"type":"string"}
# @markdown ---
destination_remote = "archive" # @param ["archive","tsd06","xuhao","haocenter","/content/drive/MyDrive/"]
destination_folder = "" # @param {"type":"string"}

# Kiểm tra và thực thi lệnh copy
if source_remote and destination_remote:
    # Chạy lệnh rclone với thông báo chi tiết
    !rclone copy --update -v --transfers 8 "{source_remote}:{input_folder}" "{destination_remote}:{destination_folder}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote.")

In [ ]:
# @title Basic: Copy Files Between Cloud Remotes
# @markdown ### Hướng dẫn:
# @markdown - **Source Remote**: Chọn hoặc nhập remote nguồn (ví dụ: `haocenter`, `/content/drive/MyDrive/`). Nếu dùng thư mục cục bộ, thêm `/` ở cuối.
# @markdown - **Input Folder**: Nhập tên thư mục/file trong remote nguồn (ví dụ: `CBZ/`, để trống nếu muốn sao chép toàn bộ remote).
# @markdown - **Destination Remote**: Chọn remote đích (ví dụ: `archive`, `tsd06`). Nếu dùng thư mục cục bộ, thêm `/` ở cuối.
# @markdown - **Destination Folder**: Nhập tên thư mục đích (ví dụ: `Backup/`, để trống nếu muốn sao chép vào root remote).
# @markdown - **Transfer Option**:
# @markdown   - `Default (--update -v)`: Sao chép với cập nhật file mới hơn, chi tiết log.
# @markdown   - `Transfers (số luồng)`: Chọn số luồng (mặc định 8, có thể nhập số khác).
# @markdown   - `S3 Concurrency (1)`: Dùng 1 luồng cho upload S3 (phù hợp với Archive).
# @markdown - Nhấn **Run** để thực thi. Nếu sai cú pháp, sẽ hiển thị lỗi và hướng dẫn.

source_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/", "haocenter", "tsd06", "xuhao", "archive"]
input_folder = "" # @param {type:"string", allow-input: true}
destination_remote = "archive" # @param ["archive", "tsd06", "xuhao", "haocenter", "/content/drive/MyDrive/"]
destination_folder = "" # @param {type:"string", allow-input: true}
transfer_option = "S3 Concurrency (1)" # @param ["Default (--update -v)", "Transfers (số luồng)", "S3 Concurrency (1)"]
transfers_value = "" # @param {type:"string", allow-input: true}

# Xử lý tùy chọn chuyển dữ liệu
transfer_flags = ""
if transfer_option == "Default (--update -v)":
    transfer_flags = "--update -v"
elif transfer_option == "Transfers (số luồng)":
    try:
        transfers_value = int(transfers_value)
        if transfers_value > 0:
            transfer_flags = f"--transfers {transfers_value} -v"
        else:
            print("Lỗi: Số luồng phải lớn hơn 0. Mặc định dùng 8.")
            transfer_flags = "--transfers 8 -v"
    except ValueError:
        print("Lỗi: Vui lòng nhập số nguyên cho Transfers. Mặc định dùng 8.")
        transfer_flags = "--transfers 8 -v"
elif transfer_option == "S3 Concurrency (1)":
    transfer_flags = "--s3-upload-concurrency 1 -v"

# Xử lý đường dẫn
if source_remote and destination_remote:
    source_path = f"{source_remote}{input_folder}".rstrip("/") + "/" if input_folder else f"{source_remote}"
    dest_path = f"{destination_remote}{destination_folder}".rstrip("/") + "/" if destination_folder else f"{destination_remote}"

    # Thay đổi tùy chọn khi đích là archive
    if destination_remote == "archive":
        command_flags = transfer_flags.replace("--update", "--ignore-existing") if "--update" in transfer_flags else transfer_flags
    else:
        command_flags = transfer_flags

    # Chạy lệnh rclone
    !rclone copy {command_flags} "{source_path}" "{dest_path}"
else:
    print("Lỗi: Vui lòng nhập cả Source Remote và Destination Remote. Xem hướng dẫn ở trên!")

# 👓 **View contents**

In [ ]:
# @title Xem Nội Dung Remote
# @markdown ### Hướng dẫn:
# @markdown - **List Mode**: Chọn cách hiển thị nội dung:
# @markdown   - `ls`: Liệt kê file và kích thước (mặc định).
# @markdown   - `lsf`: Chỉ hiển thị tên file/thư mục.
# @markdown   - `lsd`: Chỉ hiển thị tên thư mục con.
# @markdown   - `size`: Hiển thị tổng kích thước của thư mục.

list_remote = "/content/drive/MyDrive/" # @param ["/content/drive/MyDrive/", "haocenter","tsd06","xuhao","archive"]
input_folder = "tinhhe-lontien-thyetmin/" # @param {"type":"string"}
list_mode = "lsf" # @param ["lsf","lsd","ls","size"]
# @markdown 🔎 Lọc file:
file_type = "mkv" # @param {"type":"string"}
file_type_sum = "Default" # @param ["Default", "Sum"]

# Xử lý file_type_sum thành lệnh phù hợp
sum_command = "" if file_type_sum == "Default" else " | wc -l"


!rclone {list_mode} "{list_remote}:{input_folder}"  | grep ".{file_type}"  {sum_command}


# ======================================
# grep ".mp4": Lọc các dòng có chứa .mp4.
# wc -l: Đếm số dòng (tức là số file .mp4).